# NFL Transfers

In [ ]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
import queries

## Build out DataFrames

In [ ]:
ranked = False
query = queries.get_query_Transfers247(ranked)
df_247 = (fx.connDBAndReturnDF(query)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

In [ ]:
year = '2004'
query = queries.get_query_UnlinkedNFL(year)
df_nfl = (fx.connDBAndReturnDF(query)).set_index('ID')
df_nfl.index.name = 'NFL_ID'
df_nfl

In [ ]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])

In [ ]:
pairs = indexer.index(df_nfl, df_247)

In [ ]:
from functions import YearNFL

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearNFL('Year', 'Year', label='Year'))
sumFields.append('Year')

In [ ]:
features = c.compute(pairs, df_nfl, df_247)

In [ ]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['Sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

In [ ]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    if (data['Sum'] > .94):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsNFLTransfer.csv")

In [ ]:
dfFinal

In [ ]:
fx.create_AnnotationFile(dfFinal, df_nfl, df_247)

In [ ]:
filename = "..//Annotations//Results/result.json"
keydataset = 3
targetkeydataset = 1
transfer = 1

fx.save_Annotations(filename, keydataset, targetkeydataset, transfer)